In [ ]:
import sys,os
sys.path.append("/home/admin/jupyter/Demo/DataSets/kfbreader/kfbreader_linux")
import kfbReader as kr
import cv2
import numpy as np
import json
import random

In [ ]:
def  get_roi(label):    # 处理 roi 区域
    with  open(label,"r") as f:
        js = json.load(f)
    rois = []
    roi = {}
    for dic in js:
        if dic["class"] ==  "roi":
            roi = dic
            roi["poses"] = []
            rois.append(roi)
        else :
            pass
    for dic in js:
        if dic["class"] ==  "roi":
            pass
        else:
            for roi1 in rois:
                if roi1["x"] <= dic["x"] and roi1["y"] <= dic["y"] and dic["x"] + dic["w"] <= roi1["x"] + roi1["w"] and dic["y"] + dic["h"] <= roi1["y"] + roi1["h"]:
                    roi1["poses"].append(dic)
    return rois

def generate_figure(pos1,roi1):  #A 按照pos随机生成一张 在roi内的图片  这个是 宽度小于 w_figure的处理
    is_in = True
    while is_in:
        if  pos1['w']<w_figure and pos1['h']<h_figure:
            random_w = random.randint(0,w_figure - pos1['w'])
            random_h = random.randint(0, h_figure - pos1['h'])
            if(pos1['x']-random_w)>=roi1['x'] and (pos1['x']-random_w+w_figure)<=roi1['x']+roi1['w'] and (pos1['y']-random_h)>=roi1['y'] and (pos1['y']-random_h+h_figure)<=roi1['y']+roi1['h']:
                figure = {'x':pos1['x']-random_w,'y':pos1['y']-random_h,'w':w_figure,'h':h_figure}
                return figure
        else:
            random_max = max(pos1['w'],pos1['h'])
            random_w = random.randint(0,random_max - pos1['w'])
            random_h = random.randint(0, random_max - pos1['h'])
            if(pos1['x']-random_w)>=roi1['x'] and (pos1['x']-random_w+random_max)<=roi1['x']+roi1['w'] and (pos1['y']-random_h)>=roi1['y'] and (pos1['y']-random_h+random_max)<=roi1['y']+roi1['h']:
                figure = {'x':pos1['x']-random_w,'y':pos1['y']-random_h,'w':random_max,'h':random_max}
                return figure
            if random_max> roi1["w"] or random_max> roi1["h"]:   ###防止 框太大
                figure = {'x':pos1['x']-1,'y':pos1['y']-1,'w':pos1['w']+2,'h':pos1['h']+2}
                return figure
            
        
def verify_not_figure(poses,figure):  #验证它不是照片。

    for pos in poses:
        if pos['x'] < figure['x'] and pos['x']+pos['w']>figure['x'] and pos['y']>=figure['y'] and pos['y']+pos['h']<=figure['y']+figure['h']:
            return True
        elif pos['x'] < figure['x'] +figure['w'] and pos['x']+pos['w']>figure['x']+figure['w'] and pos['y']>=figure['y'] and pos['y']+pos['h']<=figure['y']+figure['h']:
            return True
        elif pos['y']< figure['y'] and pos['y']+pos['h']>figure['y'] and pos['x']>=figure['x'] and pos['x']+pos['w']<=figure['x']+figure['w']:
            return True
        elif pos['y']< figure['y']+figure['h'] and pos['y']+pos['h']>figure['y']+figure['h'] and pos['x']>=figure['x'] and pos['x']+pos['w']<=figure['x']+figure['w']:
            return True
        elif pos['x'] < figure['x'] and pos['x']+pos['w']>figure['x'] and pos['y']<figure['y'] and pos['y']+pos['h']>figure['y']:
            return True
        elif pos['x'] < figure['x'] +figure['w'] and pos['x']+pos['w']>figure['x']+figure['w'] and pos['y']<figure['y'] and pos['y']+pos['h']>figure['y']:
            return  True
        elif pos['x'] < figure['x'] and pos['x']+pos['w']>figure['x'] and pos['y']<figure['y']+figure['h'] and pos['y']+pos['h']>figure['y']+figure['h']:
            return True
        elif pos['x'] < figure['x'] +figure['w'] and pos['x']+pos['w']>figure['x']+figure['w'] and pos['y']<figure['y']+figure['w'] and pos['y']+pos['h']>figure['y']+figure['w']:
            return True
        else:
            continue

    return False

def add_pos_of_figure(poses,figure ):  # 计算这张图包括的所有的pos
    figure['poses'] = []
    for pos in poses:
        if pos['x']>=figure['x'] and pos['y'] >=figure['y'] and pos['x']+pos['w']<=figure['x']+figure['w'] and pos['y']+pos['h']<=figure['y']+figure['h']:
            figure['poses'].append(pos)

    return figure

def make_one_data(figure,roi,roi1,save_name):  # 制作一张图片 和 对应的json 文件。 roi 是真正的图片
    rx_f_r = int((figure['x']-roi1['x']))  ##相对roi1的 x 坐标。
    ry_f_r = int((figure['y']-roi1['y']))
    rw_f_r = int(figure['w'])
    rh_f_r = int(figure['h'])

    cropped = roi[ ry_f_r:ry_f_r+rh_f_r,  rx_f_r:rx_f_r+rw_f_r] #注意 h,w
#     print('%d %d %d %d'%(ry_f_r,ry_f_r+rh_f_r,rx_f_r,rx_f_r+rw_f_r))
    dic = dict()
    dic['size'] = [rw_f_r, rh_f_r]   #生成图片宽，高
    dic['scale'] = scale
    dic['k'] = k
    dic['has_box'] = True
    dic['at_20_Original_image'] = [figure['x'],figure['y'],figure['w'],figure['h']]
    dic['boxes'] = []

    #画图，为了看看情况。
    for pos in figure['poses']:
        rx_f_p = int((pos['x']-figure['x'])) ##相对 figure的 pos x坐标。
        ry_f_p = int((pos['y']-figure['y']))
        rw_p = int(pos['w'])
        ry_p = int(pos['h'])

        pos_dict = dict()
        pos_dict['x'] = rx_f_p
        pos_dict['y'] = ry_f_p
        pos_dict['w'] = rw_p
        pos_dict['h'] = ry_p
        pos_dict['class'] = pos['class']
        pos_dict['at_20_Original_image'] = pos


        dic['boxes'].append(pos_dict)
#         cv2.rectangle(cropped, (rx_f_p, ry_f_p), (rx_f_p + rw_p, ry_f_p + ry_p), (0, 255, 0), 2) #只为了演示。 正真开始请注释
    
    save_tu_name = path_tu+save_name+".jpg"
    save_json_name = path_json+save_name+'.json'
    cv2.imwrite(save_tu_name, cropped)

    with open(save_json_name, 'w') as f:
        json.dump(dic, f)


In [ ]:
def  get_roi2(label,k=1):
    with  open(label,"r") as f:
        js = json.load(f)
    rois = []
    roi = {}
    for dic in js:
        dic['x'] *= k
        dic['y'] *= k
        dic['w'] *= k
        dic['h'] *= k
    for dic in js:
        if dic["class"] ==  "roi":
            roi = dic
            roi["poses"] = []
            rois.append(roi)
        else : 
            pass
    for dic in js:
        if dic["class"] ==  "roi":
            pass
        else:
            for roi1 in rois:
                if roi1["x"] <= dic["x"] and roi1["y"] <= dic["y"] and dic["x"] + dic["w"] <= roi1["x"] + roi1["w"] and dic["y"] + dic["h"] <= roi1["y"] + roi1["h"]:
                    roi1["poses"].append(dic)
    return rois

In [ ]:
#####新的策略2
import sys
sys.path.append("/home/admin/jupyter/Demo/DataSets/kfbreader/kfbreader_linux")
import kfbReader as kr
import cv2
import numpy as np
import json
import glob
import copy
scale = 20
k = 1    #都是相对20 的缩小倍数

w_figure = 1000*k
h_figure = 1000*k

###保存文件地方
path_tu = r"/home/admin/jupyter/Demo/DataSets/Data/images/"
path_json = r"/home/admin/jupyter/Demo/DataSets/Data/images_json/"
if not os.path.exists(path_tu):
    os.makedirs(path_tu)
if not os.path.exists(path_json):
    os.makedirs(path_json)
####读取的地方
path = "/home/admin/jupyter/Data/train/"
l=glob.glob(path+'*.json')
print(len(l))
for name1 in l:  
    name = os.path.basename(name1).split(".")[0]

    file1 = path + name + ".kfb"
    label1 = path + name + ".json"
    reader = kr.reader()
    reader.ReadInfo(file1, 20, False)  #  准确度一样，false效率快.  scale是这里无法设置的，都是20.

    print('name:%s width:%d height:%d scale:%d '%(name,reader.getWidth(),reader.getHeight(),reader.getReadScale()))

    rois = get_roi(label1)
    for i, roi1 in enumerate(rois):
        print('roi No.%d :%d'%(i,len(roi1['poses'])))
        roi = reader.ReadRoi(roi1["x"] //k, roi1["y"] //k, roi1["w"] //k, roi1["h"] //k, scale) # roi 是图 roi1 是字典
        if (roi1["w"]< w_figure) or (roi1["h"]< h_figure):    #防止 roi 过小
            figure = {'x':roi1["x"],'y':roi1["y"],'w':roi1["w"],'h':roi1["h"]}
            figure = add_pos_of_figure(roi1['poses'], figure)
            save_name = name + "_" + str(i) + "_0"
            print('small roi pos_num:%d,,save name:%s'%(len(figure["poses"]),save_name))
            make_one_data(figure, roi, roi1, save_name)
            continue
        if roi1["poses"][0]["class"]=="Trichomonas":  #对 滴虫的roi特殊处理
            j=0
            roi_t = copy.deepcopy(roi1["poses"])   ####注意坑
            roi_old = []
            while len(roi_t)>0:
                is_not_figure = True
                num_of_times = 0
                while is_not_figure:
                    num_of_times += 1
                    if num_of_times > 3000:    # 生成3000次，如果不成，放弃该图
                        print("艰难生产 %s %s"%(name,roi_t[0]["class"]))
                        del roi_t[0]
                        break
                    figure = generate_figure(roi_t[0], roi1)  # 生成一张图              
                    if verify_not_figure(roi1['poses'], figure):  #验证这张图片是否合格
                        continue
                    else:
                        is_not_figure = False                         #合格跳出循环
                        figure = add_pos_of_figure(roi1['poses'], figure) # 处理这张图包括的 pos

                        for f_pos in figure['poses']:   ###这步是特殊的。                       
                            if f_pos in roi_old:
                                is_not_figure = True  #不行，接着生成。
                                break

                        if is_not_figure:
                            pass
                        else:
                            for f_pos in figure['poses']:
                                if f_pos in roi_t:   
                                    roi_old.append(f_pos)
                                    roi_t.remove(f_pos)
                            save_name = name + "_" + str(j)+"_0"
                            print('figure pos_num:%d,save name:%s'%(len(figure["poses"]),save_name))
                            make_one_data(figure, roi, roi1, save_name)  #保存一张图片
                            j=j+1
            continue

        for j, pos1 in enumerate(roi1['poses']):
            if(pos1['w']>1200)or(pos1['h']>1200): #过滤掉太大的样本
                continue
            num_ci = 1
            if pos1['class'] == "Candida":  #对 念珠菌 样本加强
                num_ci = 5
            while num_ci>0 :
                is_not_figure = True
                num_of_times = 0
                while is_not_figure:
                    num_of_times += 1
                    if num_of_times > 3000:    # 生成3000次，如果不成，放弃该图
                        print("艰难生产 %s %s"%(name,pos1["class"]))
                        break
                    figure = generate_figure(pos1, roi1)  # 生成一张图
                    if verify_not_figure(roi1['poses'], figure):  #验证这张图片是否合格
                        continue
                    else:
                        is_not_figure = False                         #合格跳出循环
                        figure = add_pos_of_figure(roi1['poses'], figure) # 处理这张图包括的 pos
                        save_name = name +"_" + str(i)+ "_" + str(j)+'_'+str(num_ci)
                        print('figure pos_num:%d,save name:%s'%(len(figure["poses"]),save_name))
                        make_one_data(figure, roi, roi1, save_name)  #保存一张图片
                num_ci-=1

In [ ]:
##### 制作 微型 数据库  small    samll_small
import sys,os
sys.path.append("/home/admin/jupyter/Demo/DataSets/kfbreader/kfbreader_linux")
import kfbReader as kr
import cv2
import numpy as np
import json
import glob
import copy
scale = 20
k = 1    #都是相对20 的缩小倍数

w_figure = 1000*k
h_figure = 1000*k

###保存文件地方
path_tu = r"/home/admin/jupyter/Demo/DataSets/Data/images_small_small/"
path_json = r"/home/admin/jupyter/Demo/DataSets/Data/images_small_small_json/"
if not os.path.exists(path_tu):
    os.makedirs(path_tu)
if not os.path.exists(path_json):
    os.makedirs(path_json)
####读取的地方
path = "/home/admin/jupyter/Data/train/"
l=glob.glob(path+'*.json')
print(len(l))
for name1 in l:  
    name = os.path.basename(name1).split(".")[0]

    file1 = path + name + ".kfb"
    label1 = path + name + ".json"
    reader = kr.reader()
    reader.ReadInfo(file1, 20, False)  #  准确度一样，false效率快.  scale是这里无法设置的，都是20.

    print('name:%s width:%d height:%d scale:%d '%(name,reader.getWidth(),reader.getHeight(),reader.getReadScale()))

    rois = get_roi(label1)
    for i, roi1 in enumerate(rois):
        print('roi No.%d :%d'%(i,len(roi1['poses'])))
        roi = reader.ReadRoi(roi1["x"] //k, roi1["y"] //k, roi1["w"] //k, roi1["h"] //k, scale) # roi 是图 roi1 是字典
        if (roi1["w"]< w_figure) or (roi1["h"]< h_figure):    #防止 roi 过小
            figure = {'x':roi1["x"],'y':roi1["y"],'w':roi1["w"],'h':roi1["h"]}
            figure = add_pos_of_figure(roi1['poses'], figure)
            save_name = name + "_" + str(i) + "_0"
            make_one_data(figure, roi, roi1, save_name)
            continue
        #新的生成图片方式
        j=0
        roi_t = copy.deepcopy(roi1["poses"])   ####注意坑
        roi_old = []
        while len(roi_t)>0:
            ####过滤一部分 这里改动了
            if roi_t[0]["class"]=="ASC-H" or roi_t[0]["class"]=="ASC-US" or roi_t[0]["class"]=="Trichomonas":
                ra = np.random.rand()
                if ra< 0.9 :     ### small = 0.7
                    del roi_t[0]
                    continue
            if roi_t[0]["class"]=="LSIL" :
                ra = np.random.rand()
                if ra< 0.2 :
                    del roi_t[0]
                    continue
            if(roi_t[0]['w']>1200)or(roi_t[0]['h']>1200): #过滤掉太大的样本
                del roi_t[0]
                continue
            is_not_figure = True
            num_of_times = 0
            while is_not_figure:
                num_of_times += 1
                if num_of_times > 3000:    # 生成3000次，如果不成，放弃该图
                    print("艰难生产 %s %s"%(name,roi_t[0]["class"]))
                    del roi_t[0]
                    break
                figure = generate_figure(roi_t[0], roi1)  # 生成一张图              
                if verify_not_figure(roi1['poses'], figure):  #验证这张图片是否合格
                    continue
                else:
                    is_not_figure = False                         #合格跳出循环
                    figure = add_pos_of_figure(roi1['poses'], figure) # 处理这张图包括的 pos

                    for f_pos in figure['poses']:   ###这步是特殊的。                       
                        if f_pos in roi_old:
                            is_not_figure = True  #不行，接着生成。
                            break

                    if is_not_figure:
                        pass
                    else:
                        for f_pos in figure['poses']:
                            if f_pos in roi_t:   
                                roi_old.append(f_pos)
                                roi_t.remove(f_pos)
                        save_name = name + "_" + str(i)+ "_" + str(j)
                        make_one_data(figure, roi, roi1, save_name)  #保存一张图片
                        j=j+1  


In [ ]:
#####剪切 no pos 的图片
import random
import sys,os
sys.path.append("/home/admin/jupyter/Demo/DataSets/kfbreader/kfbreader_linux")
import kfbReader as kr
import cv2
import json

###保存文件地方
path_tu = r"/home/admin/jupyter/Demo/DataSets/Data/images_small_small/"
path_json = r"/home/admin/jupyter/Demo/DataSets/Data/images_small_small_json/"
if not os.path.exists(path_tu):
    os.makedirs(path_tu)
if not os.path.exists(path_json):
    os.makedirs(path_json)
###读取路径
path = "/home/admin/jupyter/Data/train/"

scale = 20
k = 1    #都是相对20的缩小倍数
###生成图像尺寸
w = 1000
h = 1000

pos_names = []
neg_names = []
for name in os.listdir(path)[1:]:
    name1,tail = name.split(".")
    if tail =="kfb":
        continue
    pos_names.append(name1+".kfb")
for name in os.listdir(path)[1:]:
    name1,tail = name.split(".")
    if tail =="json":
        continue
    if name not in pos_names:
        neg_names.append(name)
print ("neg文件个数",len(neg_names))

for name in neg_names:
    file1 = path  + name
    reader = kr.reader()
    reader.ReadInfo(file1, 20, False)

    print('width:', reader.getWidth())
    print('height', reader.getHeight())
    print('scale', reader.getReadScale())

    i = 0
    while i<2 :
        random_x = random.randint(5000, reader.getWidth() - 5000-w)
        random_y = random.randint(5000, reader.getHeight() - 5000 -w)

        roi = reader.ReadRoi(random_x, random_y, w, h, scale)

        dic = dict()
        dic['size'] = [w, h]  # 生成图片宽，高
        dic['scale'] = scale
        dic['k'] = k
        dic['has_box'] = False
        dic['at_20_Original_image'] = [random_x, random_y, w, h]
        dic['boxes'] = []

        save_name = name+"_0"+"_"+str(i)
        save_tu_name = path_tu + save_name + ".jpg"
        save_json_name = path_json + save_name + '.json'
        cv2.imwrite(save_tu_name, roi)
        with open(save_json_name, 'w') as f:
            json.dump(dic, f)

        i += 1

    ### A 边上生成方式

#     random_x1 = random.randint(0, 4400)
#     random_y1 = random.randint(0, 4400)

#     roi = reader.ReadRoi(random_x1, random_y1, w, h, scale)

#     dic = dict()
#     dic['size'] = [w, h]  # 生成图片宽，高
#     dic['scale'] = scale
#     dic['k'] = k
#     dic['has_box'] = False
#     dic['at_20_Original_image'] = [random_x1, random_y1, w, h]
#     dic['boxes'] = []

#     save_name = name + "_1" + "_0"

#     save_tu_name = path_tu + save_name + ".jpg"
#     save_json_name = path_json + save_name + '.json'
#     cv2.imwrite(save_tu_name, roi)
#     with open(save_json_name, 'w') as f:
#         json.dump(dic, f)


In [ ]:
###切割 test 集合
import random
import sys,os
sys.path.append("/home/admin/jupyter/Demo/DataSets/kfbreader/kfbreader_linux")
import kfbReader as kr
import cv2
import json
import numpy as np 

scale = 20
k = 1    #都是相对20 的缩小倍数
total=0
w_figure = 1000*k
h_figure = 1000*k

###保存文件地方
path_tu = r"/home/admin/jupyter/Demo/DataSets/Data/images_test/"
path_json = r"/home/admin/jupyter/Demo/DataSets/Data/images_test_json/"
if not os.path.exists(path_tu):
    os.makedirs(path_tu)
if not os.path.exists(path_json):
    os.makedirs(path_json)
#####
path = "/home/admin/jupyter/Data/test/"
########
for name1 in os.listdir(path)[2:]:
    name2,tail = name1.split(".")
    if tail =="kfb": 
        continue
    name = name2
    file1 = path + name + ".kfb"
    label1 = path + name + ".json"
    reader = kr.reader()
    reader.ReadInfo(file1, 20, False)  #  准确度一样，false效率快.  scale是这里无法设置的，都是20.
    print('image:',name)
    print('width:', reader.getWidth())
    print('height', reader.getHeight())
    print('scale', reader.getReadScale())

    with  open(label1, "r") as f: #！！！
        js = json.load(f) 
    for i,j in enumerate(js):
        if j['class']!='roi':
            print("something wrong")
        if j['w']<w_figure or j['h']<w_figure:
            
            dic = dict()
            dic['size'] = [j['w'], j['h']]  # 生成图片宽，高
            dic['Original_name'] = name
            dic['overlapping'] = '0'
            dic['scale'] = 20
            dic['k'] = k
            roi = reader.ReadRoi(j['x'], j['y'], j['w'],j['h'], scale)
            dic['xy'] = [j['x'],j['y']]
            save_name = name + "_0"  + "_"+str(i) 
            cv2.imwrite(path_tu + save_name + '.jpg', roi)
            with open(path_json + save_name + ".json", 'w') as f:
                json.dump(dic, f)
            print("小图片: %s,%s"%(dic['xy'],dic['size']))
            total+=1
        else:
            x_list = np.arange(j['x'],j['x']+j['w'],500)  # 500 是步长
            y_list = np.arange(j['y'],j['y']+j['h'],500) #！！！

            x_list = x_list[np.where(x_list<j['x']+j['w']-w_figure)]
            x_list = np.append(x_list,j['x']+j['w']-w_figure)
            y_list = y_list[np.where(y_list<j['y']+j['h']-h_figure)]
            y_list = np.append(y_list,j['y']+j['h']-h_figure)

            dic = dict()
            dic['size'] = [w_figure, h_figure]  # 生成图片宽，高
            dic['Original_name'] = name
            dic['overlapping'] = '500'
            dic['scale'] = 20
            dic['k'] = k

            for n,x in enumerate(x_list):
                for m,y in enumerate(y_list):
                    roi = reader.ReadRoi(int(x), int(y), w_figure, h_figure, scale) ##注意 numpy 的 int 和 真正int的区别 
                    dic['xy'] = [int(x), int(y)]
                    save_name = name + "_" + str(i)+"_"+ str(m) + "_" + str(n) #！！！
                    cv2.imwrite(path_tu + save_name + '.jpg', roi)
                    with open(path_json + save_name + ".json", 'w') as f:
                        json.dump(dic, f)
            t=len(x_list)*len(y_list)
            total+=t
            print('no.%d roi:%s %s %s %s total:%d'%(i,j['x'],j['y'],j['w'],j['h'],t))
    print('total:',total)